Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

**IMPORTANT: DO NOT COPY OR SPLIT CELLS.** If you do, you'll mess the autograder. If need more cells to work or test things out, create a new cell. You may add as many new cells as you need.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and group below:

In [ ]:
COURSE = "StatModels_2020_q3"
GROUP = "" # Either D2A or D2B
NAME = "" # Match your GitHub Classroom ID

---

#### Gonzalo G. Peraza Mues, 2020

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Optimation Basics

Many problems in statistics and machine learning can be framed as optimization problems, i.e., finding the right set of parameter values which maximizes or minimizes a function. Optimization theory is very mathematical in nature, involving the fields of multivariate calculus, linear algebra, differential equations, computer science, etc. It is also a field of very active research, many of which is focused on finding ways to efficiently train deep neural networks. 

Here, we will cover the very basics of optimization, and demonstrate some practical applications. Numerical optimization methods will serve us along the rest of the course, from distribution fitting up to Monte Carlo methods. Though we will not concern ourselves too much about the details of the optimization algorithms, it is important to understand, at least in broad terms, what is happening under the hood when, for example, we fit a distribution by maximizing the likelihood function.

# Convex vs non-convex functions

We start the discussion with single variable functions of the form $y=f(x)$. We are interested in finding the minimum value of a function. In what concerns us, we can distinguish two kind of functions, those with a single minimum (global minimum) and those with multiple minima (possible several local minima, and a global minimum). The former are called *convex functions*, while the latter are called *non-convex*. The distinction is important because many optimization methods can only guarantee finding a local minimum, and, if the function is *non-convex*, the global minimum may elude us. Since convex functions possess only one minimum, the global minimum, we can be sure the algorithm will converge to it. On the other hand, trying to find the global minimum of non-convex function can be prohibitively expensive (computationally speaking).

A convex is convex if $f(x)$ is above all of its tangents, or, equivalently, for two points $A$ and $B$, $f(C)$ lies below the segment $[f(A), f(B)]$, for $A < C < B$. It can be proven that for a convex function a local minimum is also a global minimum. Then, in some sense, the minimum is unique.

![A convex function. From: https://scipy-lectures.org/_images/sphx_glr_plot_convex_001.png](images/convex_f.png)

This is not true for a non-convex function, notice the two minima.

![A non-convex function. From: https://scipy-lectures.org/_images/sphx_glr_plot_convex_002.png](images/non-convex_f.png)


# Minimizing without derivatives.

## Brent's method for single variable functions

Functions of one variable can be minimized  without derivatives by employing smart search algorithms. The most used algorithm to minimize scalar functions is the Brent's method. Let's consider the following example, from [this reference](https://www.wiwi.uni-wuerzburg.de/fileadmin/12010500/user_upload/skripte/ss10/CompEcon/cechap33.pdf):

A household can consume two goods $x_1$ and $x_2$. He values the consumption
of those goods with the joint utility function
$$
u(x_1, x_2) = x_1^{0.4}+(1+x_2)^{0.5}
$$

Here $x_2$ acts as a luxury good, i.e. the household will only consume $x_2$, if his available
resources $W$ are large enough. $x_1$ on the other hand is a normal good and will always be
consumed. Naturally, we have to assume that $x_1, x_2 \geq 0$. With the prices for the goods
being $p_1$ and $p_2$, the households has to solve the optimization problem
$$
\underset{x_1,x_2\geq 0}{\text{max}} u(x_1, x_2) = x_1^{0.4}+(1+x_2)^{0.5}
\quad\text{s.t.}\quad p_1 x_1 + p_2 x_2 = W.
$$
Note that there is no analytical solution to this problem.

First, note that this seems like a problem of two variables $x_1$ and $x_2$, but we can use the constrain to solve for $x_1$ and plug it back into the utility function, transforming the two-variable constrained problem into the unconstrained single variable problem:
$$
\underset{x_2\geq 0}{\text{max}} \left[ \frac{W-p_2 x_2}{p_1} \right]^{0.4} + (1 + x_2)^{0.5}.
$$
So, we have a maximization problem... But, we were talking about minimizing functions. That's fine, it turns out that maximizing a function $f(x)$ is equivalent to minimizing $-f(x)$. We just need to change the sign.

In [ ]:
# Define the negative utility function.
def u(x2, p1, p2, W):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Visualize the function.
x_grid = np.linspace(0, 5, 1000)
plt.plot(x_grid, [u(x2, 1, 2, 10) for x2 in x_grid])
plt.xlabel('$x_2$')
plt.ylabel('$u(x_2)$')
### BEGIN HIDDENT TESTS
assert round(u(3, 1, 2, 10), 10) == -3.7411011266

Bret's method relies on a parabolic approximation of the actual function $f(x)$. Finding the minimum of a parabola is quite easy. If the parabola is given by $a + bx + cx^2$, the its minimum is located at
$$
x = -\frac{b}{2c}.
$$
The following figure shows how Brent's method proceeds in finding a minimum.
![Brent's method.](images/brent.png)

We start with the initial interval $[a, b]$ and compute the intersection point $x_1 = (a+b)/2$. We then compute a parabola that exactly contains the three points $(a, f(a))$, $(b, f(b))$ and $(x_1, f(x_1))$.The
minimum of this parabola can be calculated and is denoted with $x_2$. If 

We then replace $b$ with $x_2$ and again compute a parabola through our new points. The method is repeated until we reach convergence.

If the parabolic approximation approach starts to converge too slowly, or the minimum leaves the desired interval, Brent's method switches to a typically slower but guaranteed method called the Golden-Search method. The Golden-Search method minimizes a one-dimensional function on the initially defined
interval $[a, b]$. The ideas behind this method is quite similar to the one of bisection search. Golden-Search however divides the interval $[a_i, b_i]$ into two sub-intervals by using the points $x_{i,1}$ and $x_{i,2}$ with
$$
x_{i,j} = a_i + \alpha_j(b_i - a_i)\quad\text{with}\quad
\alpha_1 = \frac{3-\sqrt{5}}{2}\text{ and }\alpha_2 = 1 - \alpha_1 = \frac{\sqrt{5} - 1}{2}.
$$
The values $\alpha_1$ and $\alpha_2$ are chosen in a way, such that the interval $[a_i, b_i]$ is intersected according to the golden ratio. We now compute the function values $f(x_{i,j})$ for $j = 1, 2$ and compare them. The next iteration's interval is then chosen according to
$$
\begin{cases}
a_{i+1} = a_i\text{ and }b_{i+1}=x_{i,2} & \text{if } f(x_{i,1} < f(x_{i,2})\\
a_{i+1} = x_{i,1}\text{ and }b_{i+1}=b_{i} & \text{otherwise}
\end{cases}
$$

The idea behind this iteration rule is quite simple. If $f(x_{i,1}) < f(x_{i,2})$, the lower values of
$f$ will be located near $x_{i,1}$, not $x_{i,2}$. Consequently, one chooses the interval $[a_i, x_{i,2}]$ as new iteration interval and therefore rules out the greater values of $f$.

![Brent's method.](images/golden.png)

Brent's method is implement in Scipy's optimize module, as default method of the `minimize_scalar` function. Let's use it:    

In [ ]:
from scipy import optimize

result = optimize.minimize_scalar(u, bracket=(0, 2.5, 5), args=(1, 2, 10))

The optional bracket argument takes a 3-element tuple (a, b, c) with $f(b) < f(a), f(c)$, as to guarantee a minimum is located in the interval $[a, c]$. The result is stored in a `OptimizeResult`, in which, among other information, one can find the optimal value of $x$.

In [ ]:
type(result)

In [ ]:
x_opt = result.x
print(x_opt)

x_grid = np.linspace(0, 5, 1000)
plt.plot(x_grid, [u(x2, 1, 2, 10) for x2 in x_grid])
plt.xlabel('$x_2$')
plt.ylabel('$u(x_2)$')
plt.plot(x_opt, u(x_opt, 1, 2, 10), 'ro');

#### **Exercise**

Explore the results object. Besides `x`, what else is stored there? Extract the number of iterations and the number of evaluations into the `iters` and `evals` variables.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
### BEGIN HIDDEN TEST
assert iters == result.nit
assert evals == result.nfev
### END HIDDEN TEST

The algorithm finds the minimum inside the specified interval in about 11 iterations. Since the negative utility is a convex function, we were able to find the global minimum, which is harder to find in non-convex functions.

Now, let's test Brent's method using non-convex functions. We start by defining the function
$$
(x - e^{-1})^2 + \epsilon e^{-5(x - 0.5 - e^{-1})^2}
$$
For $\epsilon=0$ the function is convex, while for $\epsilon \neq 0$ it becomes non-convex.

In [ ]:
def f(x, epsilon):
    x_0 = np.exp(-1)
    return (x - x_0)**2 + epsilon*np.exp(-5*(x - .5 - x_0)**2)

x = np.linspace(-1, 3, 100)
for eps in range(10):
    plt.plot(x, f(x, eps), label=f'$\epsilon={eps}$')
plt.legend();

#### **Exercise**:

We want to test how the value of $\epsilon$ influences the convergence of the Brent's method. Create a for loop to minimize the different functions above. Store the minimum values found into the `x_min` list. Plot each function together with its minimum. Use the bracket (-2, 2, 4.5). Visually check which optimization got stuck a local minimum larger than the global.

In [ ]:
x_min = []
for eps in range(10):
    # YOUR CODE HERE
    raise NotImplementedError()
plt.legend();

In [ ]:
### BEGGIN HIDDEN TESTS
assert np.allclose(x_min, [0.3678794411714423, 0.12812912819302427, 0.05960911516130499, 0.01956411505088859, 
                    -0.008570917021825646, -0.03017584988638197, 1.6289592048571786, 1.650344520836931, 
                    1.6683276953386315, 1.683807271808013])

So, at least for some values of $\epsilon$ (the larger ones), and using that bracket, the optimization gets trapped in a local minimum. Try changing the bracketing option in order to assert the global minimum is found in all cases. Do this is a new cell as to not change the solution to the previous exercise.

In general, searching for the global minimum is a very expensive computational effort, and an active area of current research. 

## Multi-variate functions

Typically, we will be interested in optimizing functions of more than one variable. A multivariate function is a function that takes a vector of values as an input and return a single scalar as an output. Mathematically, they are described as $f:\mathbb{R}^n \rightarrow \mathbb{R}$. For example, the two-variable function
$$
f(x, y) = x^2 + y^2 + x \sin{y}+ y \sin{x}
$$
takes as input the vector $[x, y]^T$, and outputs a single value $f(x,y)$. We can visualize two-variable functions using Matplotlib's 3D plotting capabilities. To plot the function $f(x,y)$ as a 3D surface we can do:

In [ ]:
# Define the function
def f(x, y):
    return x**2 + y**2 + x*np.sin(y) + y*np.sin(x)

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')

# Create grid values along the two variables
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)

# Meshgrid creates two matrices with X and Y coordinates
X, Y = np.meshgrid(X, Y)

# Evaluate function on grid
Z = f(X, Y)

ax.plot_surface(X, Y, Z, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x,y)')
plt.tight_layout();

If you have Ipython Widgets configured we can make the plot interactive:

In [ ]:
%matplotlib widget

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x,y)');

Another useful way of visualizing two-variable functions is using contour plots. A contour plot is an 2D projection of two-variable function into the x-y plane. Lines (contours) in the contour plot define points of equal height, i.e., the function remains constant along these lines. 

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,6))
# Draw the filled contours
plt.contourf(X, Y, Z, levels=10, alpha=0.75)
# Draw the contour lines
C = plt.contour(X, Y, Z, levels=10, colors='black')
# Add height labels
plt.clabel(C, inline=1, fontsize=10);

## Nelder-Mead algorithm for multi-variable functions

Now, onto the Nelder-Mead algorithm. The Nelder-Mead method uses a geometrical shape called a simplex as its 'vehicle' of sorts to search the domain. This is why the technique is also called the Simplex search method. In layman's terms, a simplex is the n-dimensional version of a 'triangle'. For 1 dimension, its a line, for 2 dimensions, its a triangle, for 3 dimensions, its a tetrahedron, etc. The shape doesn't have to be symmetrical/equilateral in any way. Note that an n-dimensional simplex has (n+1) vertexes.

The method does not require any derivative information, which makes it suitable for problems with non-smooth functions. It is widely used to solve parameter estimation and similar statistical problems, where the function values are uncertain or subject to noise. It can also be used for problems with discontinuous functions, which occur frequently in statistics and experimental mathematics.

We start by considering the 1D case, for which the simplex is a line. One simple thing to try would be to sample two points relatively near each other, and just repeatedly take a step down away from the largest value. The obvious problem in this approach is using a fixed step size: it can't get closer to the true minima than the step size so it doesn't converge. It also spends too much time inching towards the minima when it's clear that the step size should be larger.

To overcome these problems, the Nelder-Mead method dynamically adjusts the step size based off the loss of the new point. If the new point is better than any previously seen value, it expands the step size to accelerate towards the bottom. Likewise if the new point is worse it contracts the step size to converge around the minima. The algorithm is implemented in the `optimize.minimize` function. To use it, you need to pass the option `method="Nelder-Mead"` to `minimize`. Let's test in a non-trivial function:
$$
f_1(x) = \log \left(1 + \vert x\vert^{2 + \sin{x}}\right)\\
$$

In [ ]:
def f1(x):
    return np.log(1 + np.abs(x)**(2 + np.sin(x)))

In [ ]:
%matplotlib inline
# Minimize requires to specify an initial guess or starting point x0
result = optimize.minimize(f1, x0=-4, method="Nelder-Mead")
print(result)

# Visualize the function
x_min = result.x
x_grid = np.linspace(-5, 6, 100)
plt.plot(x_grid, f1(x_grid))
plt.plot(x_min, f1(x_min), 'or');

Let's try to visualize the algorithm in 1D. As you can verify, the results object contains the simplex at the final iteration, in this case a line defined by two points. What we want is to the simplex at each iteration to visualize how the simplex evolves through the optimization process. In order to do this, we are going to forcibly terminate the algorithm at a specified number of iterations until convergence at store the simplex at each step.

The following function provides a visualization for the Nelder-Mead algorithm in 1D. The `final_simplex` tuple of the results object contains two arrays, the first one contains the x-values of the points of the simplex, similarly, the second one contains the y-values. Animation is achieved by repeating and clearing the plot each iteration. Study the function, as you will need to modify it for two-dimensional functions later on.

In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
from time import sleep

def visualize_opt_1d(func, interval, niter, method='Nelder-Mead'):
    """ A visualization for the Nedler-Mead algorithm in 1D. It allows to explore each iteration of the algorithm.
    INPUT:
        - func: the function to minimize
        - interval: interval to visualize, must contain the starting point x0, of the form (a, x0, b)
        - niter: number of iterations to perform
    OUPUT:
        - An step by step visualization.
    """
    
    # Unpack interval
    a, x0, b = interval
    assert a < x0 < b
    
    # Initialize figure
    fig = plt.figure()
    x_grid = np.linspace(a, b, 500)
    
    # Apply method. To have access to the iteration, do this in an
    # artificial way: allow the algorithm to iter only once
    converged = False
    for i in range(niter):
        # Run optimizer
        result = optimize.minimize(func, x0, method=method, options={"maxiter": i})
        
        # Store the current simplex.
        clear_output(wait=True)
        plt.plot(x_grid, func(x_grid))
        simplex_x = result.final_simplex[0].ravel()
        simplex_y = result.final_simplex[1].ravel()
        plt.plot(simplex_x, simplex_y, 'o-', color='r')
        plt.show()
        sleep(0.4)
        
        # Check if already converged
        if result.success:
            print(f'Converged at {i} iterations')
            converged = True
            break
    if not converged:
        print(f'Did not converge after {niter} iterations.')

In [ ]:
%matplotlib inline
visualize_opt_1d(f1, interval=(-5, -4, 6), niter=30)

Try the following functions with the Nedler-Mead algorithm. One has many local minima, the other has discontinuous derivatives. Implement each function and visualize the algorithm in a suitable interval. Create more cells as needed.
$$
f_2(x) = \left(2 + \frac{\sin 50 x}{50}\right)\left(\arctan x\right)^2\\
f_3(x) = \left\vert \left \lfloor{x}\right \rfloor  - 50 \right\vert
$$

In [ ]:
def f2(x):
    return (2 + np.sin(50*x)/50)*(np.arctan(x))**2

def f3(x):
    return np.abs(np.floor(x) - 50)

In [ ]:
# Visualize f2
visualize_opt_1d(f2, interval=(-5, -4, 6), niter=30)

In [ ]:
# Visualize f3
visualize_opt_1d(f3, interval=(30, 35, 70), niter=30)

To understand how the algorithm works in more than one dimension, we first need to discuss the simplex transformations in detail. The main idea, is that the simplex evolves at each iteration in order to explore the landscape of the function in a directed search towards the minimum. The simplex adapts itself to the local landscape, elongating down long inclined planes, changing direction on encountering a valley at an angle, and contracting in the neighborhood of a minimum.

Nelder-Mead starts off with a randomly-generated simplex. At every iteration, it proceeds to reshape/move this simplex, one vertex at a time, towards an optimal region in the search space. During each step, it basically tries out one or a few modifications to the current simplex, and chooses one that shifts it towards a 'better' region of the domain. In an ideal case, the last few iterations of this algorithm would involve the simplex shrinking inwards towards the best point inside it. At the end, the vertex of the simplex that yields that most optimal objective value, is returned. The general algorithm is given below.

- Construct the initial working simplex S using n+1 points (n is the number of dimensions) around the initial guess $x_0$. The exact method of creating the initial simplex is implementation dependent.
- Repeat the following steps until the termination test is satisfied:
  - calculate the termination test information;
  - if the termination test is not satisfied then transform the working simplex.
- Return the best vertex of the current simplex S and the associated function value.

One iteration of the Nelder-Mead method consists of the following three steps:
1. **Ordering**. All the points are ordered/sorted, such that the value of $f$ for the first point is the highest, and that for the last point is the lowest. Let the indices of the first(worst), second(second worst) and last(best) points be $h$, $s$, $l$ respectively.

2. **Computing the Centroid**. Consider all points except the worst ($x_h$). Compute their centroid(mean) as:
$$
c = \frac{1}{n}\sum_{i\neq h} x_i
$$

3. **Transformation**. This is the most important part of the whole process – transforming the simplex. Transformation takes place in one of the following ways:
  1. **Reflection**. This is the first type of transformation that is tried out. You compute the 'reflected' point as:
  $$
  x_r = c + \alpha(c - x_h)
  $$
  $\alpha$ is called the reflection parameter, and is usually equal to one. $x_r$ is essentially a point on the line joining $c$ and $x_h$, but located away from $x_h$. This is done in an attempt to move the simplex in a direction thats away from the sub-optimal region around $x_h$. For a 2-D problem, heres what reflection looks like:
  
  ![](images/neldermead_1.jpg)
  
  Now, if $f(x_s) > f(x_r) \geq f(x_l)$ -- that is, that $x_r$ is better than the second-worst point, but not better than the current best, we just replace $x_h$ with $x_r$ in the simplex, and move on to the next iteration.
  
  2. **Expansion**. If our luck is great and the reflected point $x_r$ happens to be better than the current best ($f(x_r) < f(x_l)$), we try to explore the possibility further.  In other words, we move a little bit more in the direction of $x_r$ from c, in the hope of finding an even better solution. The expanded point is defined as:
  $$
  x_e = c + \gamma(x_r - c)
  $$
  \gamma is called the expansion parameter, and its value in most implementations is 2. Heres what 2-D Expansion looks like:
  
  ![](images/neldermead_2.jpg)
  
  We then replace $x_h$ with the better of the two points: $x_e$ and $x_r$ in the simplex. This is called 'greedy optimization' -- we have replaced the worst point with the better of the two options we had. Theres another variant called 'greedy expansion', which replaces $x_h$ with $x_e$, as long as $f(x_e) < f(x_l)$. This is done irrespective of whether $x_e$ is better than $x_r$. The intuition for this, comes from the fact that Expansion always leads to a bigger simplex, which means more exploration of the search space.
  
  3. **Contraction**. Suppose our reflection point was worst than $x_s$ (i.e. the second worst point). In that case, we need to realize that the direction defined by $x_r$ may not be the ideal one for moving. Therefore, we end up contracting our simplex. The contraction point x_c is defined as:
  $$
  x_c = c + \beta(\min(x_h, x_r) - c)
  $$
  The simplex contracts respect to the better of $x_h$ or $x_r$. $\beta$ is called the contraction parameter, and is usually equal to 0.5. In essence, we try to go against our exploration, instead contracting the simplex. An 'inward' 2-D contraction looks like this: 

  ![](images/neldermead_3.jpg)
  
  If $f(x_c) < f(\min(x_h, x_r))$, which means that the contracted point is better than the current-worst, then we replace $x_h$ with $x_c$ in the simplex. However, if this is not the case, then we go to our last resort: the Shrink contraction.
  
  4. **Shrink contraction**. In this case, we redefine the entire simplex. We only keep the best point ($x_l$), and define all others with respect to it and the previous points. The j$^{th}$ new point, will now be defined as:
  $$
  x_j = x_l + \delta (x_j - x_l)
  $$
  \delta is called the shrinkage parameter, and is equal to 0.5 in most implementations. What we are essentially doing with the above definition, is moving each point in the simplex towards the current best point, in the hope of converging onto the best neighborhood. A 2-D Shrinking transformation looks like:
  
  ![](images/neldermead_5.jpg)
  
4. **Termination**. Termination is usually reached when:

  1. A given number of iterations is reached
  2. The simplex reaches some minimum 'size' limit.
  3. The current best solution reaches some favorable limit.

Let's apply the algorithm to two-variable functions. We again a few test functions:
$$
\begin{align}
f_1(x, y) =& x^2 + y^2 + x\sin{y} + y\sin{x}\\
f_2(x, y) =& (x^2 + y  - 11)^2 + (x + y^2 - 7)^2\\
f_3(x, y) =& (1 - x)^2 + 100(y - x^2)^2\\
f_4(x, y) =& 0.26(x^2+y^2) + 0.48 xy
\end{align}
$$

In [ ]:
# In order to work with the minimize function, functions must accept a vector as input.

def f1(x):
    return x[0]**2 + x[1]**2 + x[0]*np.sin(x[1]) + x[1]*np.sin(x[0])

def f2(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2

def f3(x):
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

def f4(x):
    return 0.26*(x[0]**2 + x[1]**2) + 0.048*x[0]*y
    
f_list = [f1, f2, f3, f4]

In [ ]:
from matplotlib import ticker
import matplotlib.colors

# Now, lets optimize each one of them.

# Define the grid.
X = np.linspace(-6, 6, 50)
Y = np.linspace(-6, 6, 50)
X, Y = np.meshgrid(X, Y)

fig, axes = plt.subplots(2, 2, figsize=(20,12))

for f, ax in zip(f_list, axes.ravel()):
    # This step is crucial, take your time understanding it, decompose it in new cells if needed.
    Z = f(np.stack([X.ravel(), Y.ravel()], axis=0)).reshape(X.shape)
    
    # Plotting the minimum
    # This a hack to represent the levels in logaritmically distributed space, so we may visualize the mins better.
    # Not that important.
    levs = np.geomspace(0.001, Z.max() - Z.min(), 20) + Z.min()

    ax.contourf(X, Y, Z, levels=levs, alpha=0.75)
    result = optimize.minimize(f, x0=[-4, 4], method="Nelder-Mead")
    ax.plot(result.x[0], result.x[1], 'ro')

In [ ]:
result.final_simplex[0][:,0]

#### **Exercise**

Modify the 1D visualization function in order to visualize the algorithm in 2D spaces. Now the simplex is a triangle and the final_simplex object is more complex.

In [ ]:
def visualize_opt_2d(func, xinterval, yinterval, niter, method='Nelder-Mead'):
    """ A visualization for the Nedler-Mead algorithm in 2D. It allows to explore each iteration of the algorithm.
    INPUT:
        - func: the function to minimize
        - xinterval: interval to visualize, must contain the starting point x0, of the form (xa, x0, xb)
        - yinterval: interval to visualize, must contain the starting point y0, of the form (ya, y0, yb)
        - niter: number of iterations to perform
    OUPUT:
        - An step by step visualization.
    """
    
    # Unpack interval
    xa, x0, xb = xinterval
    assert xa < x0 < xb
    ya, y0, yb = yinterval
    assert ya < y0 < yb
    v0 = [x0, y0]
    
    
    # Initialize figure, define the grid as before, X,Y, X and levs
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Apply method. To have access to the iteration, do this in an
    # artificial way: allow the algorithm to iter only once
    converged = False
    for i in range(niter):
        # Run optimizer
        # YOUR CODE HERE
        raise NotImplementedError()
        
        clear_output(wait=True)
        fig = plt.figure(figsize=(15,9))
        # Plot the current simplex
        # YOUR CODE HERE
        raise NotImplementedError()
        plt.show()
        sleep(0.4)
        
        # Check if already converged
        if result.success:
            print(f'Converged at {i} iterations')
            converged = True
            break
    if not converged:
        print(f'Did not converge after {niter} iterations.')

In [ ]:
# Test with different functions.
visualize_opt_2d(f1, xinterval=[-6,-4,6], yinterval=[-6,4,6], niter=100)

## References and textual sources:
- The Scipy Lecture Notes, at https://scipy-lectures.org/.
- https://www.wiwi.uni-wuerzburg.de/fileadmin/12010500/user_upload/skripte/ss10/CompEcon/cechap33.pdf
- https://codesachin.wordpress.com/2016/01/16/nelder-mead-optimization/
- http://www.scholarpedia.org/article/Nelder-Mead_algorithm